In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import numpy as np

C:\Users\Dpsdp\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data=pd.read_csv("rp_data.csv")
data.head(5)

,Name,Description,Type,Season,Location
0,Arugam Bay Beach,"Popular surf destination, known for its waves ...",beach,November-April,"Eastern Province, Ampara District"
1,Unawatuna Beach,Renowned for its golden sands and snorkeling o...,beach,November-April,"Southern Province, Galle District"
2,Hikkaduwa Beach,Vibrant coral reefs and nightlife make it a ho...,beach,November-April,"Southern Province, Galle District"
3,Ventura Beach,Calm waters ideal for families and water sports.,beach,November-April,"Southern Province, Galle District"
4,Mirissa Beach,Famous for whale watching and scenic beauty,beach,November-April,"Southern Province, Matara District"


In [3]:
# Convert to DataFrame
df = pd.DataFrame(data, columns=["Name", "Description", "Type", "Season", "Location"])

In [4]:
# Encoding the Type and Season columns
season_encoder = LabelEncoder()
df['Season'] = season_encoder.fit_transform(df['Season'])

type_encoder = LabelEncoder()
df['Type'] = type_encoder.fit_transform(df['Type'])

# Encoding Location
location_encoder = LabelEncoder()
df['Location'] = location_encoder.fit_transform(df['Location'])

In [5]:
# Features (input) and Target (output)
X = df[['Season', 'Type']]  # Using season and type as inputs
y = df[['Name', 'Location']]  # The output will be the Name and Location

In [6]:
# One-hot encoding for 'Name' and 'Location'
name_encoder = LabelEncoder()
df['Name'] = name_encoder.fit_transform(df['Name'])
y = pd.get_dummies(y)  # One-hot encode Name and Location

In [7]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Ensure inputs are in the correct dtype (float32) for training
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)

# Ensure targets are in the correct dtype (int32) for classification
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

In [9]:
# Build the deep learning model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

C:\Users\Dpsdp\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - accuracy: 0.0000e+00 - loss: 48.5124 - val_accuracy: 0.0000e+00 - val_loss: 46.9539
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.0000e+00 - loss: 47.2861 - val_accuracy: 0.0000e+00 - val_loss: 45.0531
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0000e+00 - loss: 47.2872 - val_accuracy: 0.0000e+00 - val_loss: 43.2857
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.0000e+00 - loss: 44.2876 - val_accuracy: 0.0000e+00 - val_loss: 41.9786
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.0000e+00 - loss: 43.6346 - val_accuracy: 0.0000e+00 - val_loss: 40.8882
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.0159 - loss: 42.1976 - val_accuracy: 0.0000e+00 - val_loss: 39.7339
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0000e+00 - loss: 40.4871 - val_accuracy: 0.0000e+00 - val_loss: 38.5875
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accurac

In [12]:
# Evaluate the model
accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy[1] * 100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7273 - loss: 35.1774
Accuracy: 72.73%


###Checking reals

In [13]:
# Step 1: Display available seasons
def get_available_seasons():
    return season_encoder.classes_


In [14]:
# Step 2: Display available types based on selected seasons
def get_matching_types(selected_seasons):
    # Filter rows by selected seasons
    selected_seasons_encoded = season_encoder.transform(selected_seasons)
    filtered_data = df[df['Season'].isin(selected_seasons_encoded)]
    return filtered_data['Type'].unique()

In [15]:
# Step 3: Get recommendations based on selected seasons and types
def recommend_places(selected_seasons, selected_types):
    # Filter dataset by selected seasons and types
    selected_seasons_encoded = season_encoder.transform(selected_seasons)
    selected_types_encoded = type_encoder.transform(selected_types)
    
    # Filter the data based on user inputs
    filtered_data = df[(df['Season'].isin(selected_seasons_encoded)) & 
                        (df['Type'].isin(selected_types_encoded))]
    
    # Make a copy of the filtered data for safe modifications
    filtered_data_copy = filtered_data.copy()
    
    # Decode Name and Location columns back to their original values
    filtered_data_copy['Name'] = name_encoder.inverse_transform(filtered_data_copy['Name'])
    filtered_data_copy['Location'] = location_encoder.inverse_transform(filtered_data_copy['Location'])
    
    # Return the decoded recommendations
    recommendations = filtered_data_copy[['Name', 'Location']]
    
    return recommendations

In [21]:
# Example of user input
seasons = get_available_seasons()  # All available seasons
print("Available Seasons: ", seasons)

# Step 2: Ask the user to select one or more seasons
selected_seasons_input = input("Please select one or more seasons (comma separated): ")
selected_seasons = [season.strip() for season in selected_seasons_input.split(',')]

# Validate selected seasons
valid_seasons = [season for season in selected_seasons if season in seasons]
if not valid_seasons:
    print("Invalid seasons selected.")
else:
    print(f"Selected Seasons: {valid_seasons}")
    
    # Step 3: Display activity types based on selected seasons
    types = get_matching_types(valid_seasons)
    decoded_types = type_encoder.inverse_transform(types)
    print(f"Available Types based on selected seasons: {decoded_types}")
    
    # Step 4: Ask the user to select one or more types
    selected_types_input = input("Please select one or more activity types (comma separated): ")
    selected_types = [type.strip() for type in selected_types_input.split(',')]

    # Validate selected types
    valid_types = [type for type in selected_types if type in decoded_types]
    if not valid_types:
        print("Invalid activity types selected.")
    else:
        print(f"Selected Types: {valid_types}")

        # Step 5: Get recommendations based on selected seasons and types
        recommendations = recommend_places(valid_seasons, valid_types)
        print("\nRecommended Places and Locations: ")
        print(recommendations)


Available Seasons:  ['Dry season' 'May-September' 'November-April' 'Year-round']
Please select one or more seasons (comma separated): November-April
Selected Seasons: ['November-April']
Available Types based on selected seasons: ['beach']
Please select one or more activity types (comma separated): beach
Selected Types: ['beach']

Recommended Places and Locations: 
               Name                                Location
0  Arugam Bay Beach       Eastern Province, Ampara District
1   Unawatuna Beach       Southern Province, Galle District
2   Hikkaduwa Beach       Southern Province, Galle District
3     Ventura Beach       Southern Province, Galle District
4     Mirissa Beach      Southern Province, Matara District
5    Nilaveli Beach  Eastern Province, Trincomalee District
6      Jungle Beach       Southern Province, Galle District
7  Hiriketiya Beach      Southern Province, Matara District
8    Pasikuda Beach   Eastern Province, Batticaloa District
9     Koggala Beach       Souther

In [17]:
model.save('trec_model.keras')

In [18]:
from tensorflow.keras.models import load_model
model = load_model('trec_model.keras')